Post-processing of ARC partitions using hypothesis testing.

In [ ]:
import pickle as pkl
import numpy as np
import OAM.OAM_functions as ov_fct
import Towlson_group_code.data_io as myFunc
import Towlson_group_code.community_detection.functional_brain_community as fbc
from tqdm import tqdm
from collections import Counter
import os

In [ ]:
def hypothesis_test(c, cycle, phase1, phase2):
    enum = {'EF': 0, 'LF': 1, 'ML': 2}
    if phase1 == 'EF':
        a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/',
                                              f'{cycle[enum[phase1]]}_auditory.pkl')
    if phase1 == 'LF':
        a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/',
                                              f'cycle_{c}_LF_partition_auditory_EF.pkl')

    b_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/',
                                          f'{cycle[enum[phase2]]}_auditory.pkl')

    group_a = myFunc.load_from_pickle('../Ovarian_hormone/pickles/cycle_stats/',
                                      f'cycle_{c}_{phase1.lower()}_stats.pkl')
    group_b = myFunc.load_from_pickle('../Ovarian_hormone/pickles/cycle_stats/',
                                      f'cycle_{c}_{phase2.lower()}_stats.pkl')

    print(len(group_a), len(group_b))
    N = 1000
    accepted = [0] * 9
    m = [0] * 9
    M = sum([0 if a_partition[n_i] == b_partition[n_i] else 1 for n_i in range(1054)])

    for n_i in tqdm(range(1054)):
        # Check if ICN assignment for n_i is the same or different between phases
        if a_partition[n_i] == b_partition[n_i]:
            continue
        else:
            m[a_partition[n_i]] += 1
            # Permutation testing - 2 p values
            a = ['A'] * N + ['B'] * N
            b = group_a[n_i] + group_b[n_i]
            p = permutation_test(a, b, a_partition[n_i], b_partition[n_i], N, num_simulations=1000)
            p = p * M

            if p < 0.05:
                # accepted += 1
                accepted[a_partition[n_i]] += 1
            else:
                # Reject. Place node back to where it was.
                b_partition[n_i] = a_partition[n_i]
    for i in range(9):
        if m[i] != 0:
            print(
                f"{ov_fct.AVG_EF_FN_AUDITORY[i]}: total of {m[i]} switching nodes, accepted {accepted[i]} moves, which is {(accepted[i] / m[i]):0.2f}%.")
        else:
            print(f"{ov_fct.AVG_EF_FN_AUDITORY[i]}: no switching nodes. {m[i]}")
    myFunc.save_to_pickle(b_partition, '../Ovarian_hormone/pickles/hypothesis_tested/',
                          f'cycle_{c}_{phase2}_partition_auditory_{phase1}.pkl')

def permutation_test(x, y, icn1, icn2, N, num_simulations=100000):
    obs_1 = Counter(y[:N])[icn1]
    obs_2 = Counter(y[N:])[icn2]
    simulated_results = 0
    for _ in (range(num_simulations)):
        s1, s2 = simulate_2_counts(x, y, icn1, icn2)
        if (s1 >= obs_1) and (s2 >= obs_2):
            simulated_results += 1
    return simulated_results / num_simulations

def simulate_2_counts(x, y, icn1, icn2):
    randomly_shuffled = np.random.permutation(x)
    mask = np.where(randomly_shuffled == 'A', True, False)
    counts = Counter(mask * y)
    r1 = counts[icn1]
    counts = Counter((~mask) * y)
    r2 = counts[icn2]
    return r1, r2

def calculate_null_distribution():
    # Set up parameters for each subject scan
    ef_input_params = myFunc.load_from_pickle("../Ovarian_hormone/pickles/individual_connectomes/",
                                              "ef_input_params.pkl")
    ef_name_to_idx = {x[0]: x[1] for x in ef_input_params.values()}

    # Load in cycles to process from spreadsheet
    # ------------------------------------------------ TEMP
    to_do = myFunc.import_XLSX('../Ovarian_hormone/data/', 'subject_level_analysis.xlsx', index_col=None,
                               sheet_name="cycles")
    # to_do = to_do.drop(columns=["ml_gamma"], axis=1)
    to_do = to_do[to_do['flag'] == 1]
    # ------------------------------------------------
    # ref_name = 'avg EF'
    # reference_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'EF_best_partition_auditory.pkl')
    for _, row in to_do.iterrows():
        c, ef_name, _, scan_name, _, _, gamma_v, _ = row
        ref_name = ef_name.strip()
        scan_name = scan_name.strip() + '.pkl'
        idx = ef_name_to_idx[ef_name.strip() + '.pkl']

        print(f"cycle: {c} | scan ", scan_name, " and chosen gamma = ", gamma_v, "Ref: ", ref_name)

        reference_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/',
                                                      f'{ref_name}_auditory.pkl')

        _ = generate_extra_partitions(idx, scan_name, c, gamma_v)
        partitions_list = generate_extra_partitions(idx, scan_name, c, gamma_v)

        stat = [[] for i in range(1054)]
        # For node i, check it's FN assignment
        for partition in tqdm(partitions_list):
            # determine ICNs
            fn_assignment, partition_norm = assign_ICNs(partition, reference_partition)

            icns = {}
            for fn, match_list in fn_assignment.items():
                for l in match_list:
                    icns[l] = fn
            fn_rev = {v: k for k, v in ov_fct.AVG_EF_FN_AUDITORY.items()}
            merged_partition = [fn_rev[icns[x]] for x in partition_norm]

            for n_i in range(1054):
                stat[n_i].append(merged_partition[n_i])

        myFunc.save_to_pickle(stat, '../Ovarian_hormone/pickles/cycle_stats/',
                              f'cycle_{c}_{scan_name[:2].lower()}_stats.pkl')

def generate_extra_partitions(idx, scan_name, c, gamma_v):
    path = f'../Ovarian_hormone/ARC/subject_level/subject_auditory/{idx}/'
    # path = f'../Ovarian_hormone/ARC/Avg_Results_auditory/'
    fname = f'{scan_name[:-4]}_extras.pkl'
    if (os.path.exists(path + fname)):
        partitions_list = myFunc.load_from_pickle(path, fname)
        print(f"Loaded existing partitions list ({len(partitions_list)})")
        return partitions_list
    else:
        if c != 'avg':
            # Re-run for 3000 iterations of community detection for chosen gamma
            with open(f'../Ovarian_hormone/pickles/individual_connectomes/{idx}/{scan_name}', 'rb') as f:
                avg_connectome = pkl.load(f)
            print(avg_connectome.shape)

            p_list, _ = fbc.get_partitions(avg_connectome, gamma=gamma_v, B='negative_asym', rep=1000)
            p_list2, _ = fbc.get_partitions(avg_connectome, gamma=gamma_v, B='negative_asym', rep=1000)
            myFunc.save_to_pickle(p_list + p_list2,
                                  path,
                                  f'{scan_name[:-4]}_extras.pkl')
            return []
        else:
            with open(f'../Ovarian_hormone/pickles/average_connectomes/averaged-{scan_name[:2]}-2022-05-09.pkl',
                      'rb') as f:
                avg_connectome = pkl.load(f)
            print(avg_connectome.shape)

            p_list, _ = fbc.get_partitions(avg_connectome, gamma=gamma_v, B='negative_asym', rep=1000)
            myFunc.save_to_pickle(p_list, path, f'{scan_name[:-4]}_extras.pkl')
            return []

def assign_ICNs(partition, reference_partition):
    reference_FN_assignment = ov_fct.AVG_EF_FN_AUDITORY
    # reference_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', f'EF_best_partition_auditory.pkl')

    ref_modules = ov_fct.get_modules(reference_partition, reference_FN_assignment)
    modules, partition_renorm = ov_fct.get_all_modules(partition)
    my_prefs = ov_fct.get_preference_list(modules, ref_modules)
    ref_prefs = ov_fct.get_preference_list(ref_modules, modules)

    match_to_ref = ov_fct.get_poly_matching(my_prefs, ref_prefs, poly_threshold=0.6)
    return match_to_ref, partition_renorm

Step 1: Generate null distributions

In [ ]:
calculate_null_distribution()

Step 2: Hypothesis test each cycle

In [ ]:
all_cycles = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'all_cycles.pkl')
for c in range(30):
    print(c, all_cycles[c])
    hypothesis_test(c, all_cycles[c], 'EF', 'LF')
    hypothesis_test(c, all_cycles[c], 'LF', 'ML')
    hypothesis_test(c, all_cycles[c], 'EF', 'ML')